# Catastrophic Forgetting Results Notebook<a name='top'/>

This notebook contains the data analysis tools and products for all the experiments associated with this project.

## Table of Contents

<a href=#setup>Setup</a>

<a href=#mnist>MNIST</a>
> <a href=#mnist_validation>Validation Phase</a><br><br>
> <a href=#mnist_testing>Testing Phase</a>

<a href=#mountain_car>Mountain Car</a>
> <a href=#mountain_car_validation>Validation Phase</a><br><br>
> <a href=#mountain_car_testing>Testing Phase</a>

<a href=#acrobot>Acrobot</a>
> <a href=#acrobot_validation>Validation Phase</a><br><br>
> <a href=#acrobot_testing>Testing Phase</a>

<a href=#summary>Summary</a>

## Setup <a name='setup'/>

In [ ]:
import acrobot.tools as ac_tools
import collections
import copy
import json
import matplotlib.pyplot as plt
import mnist.tools as mnist_tools
import mountain_car.tools as mc_tools
import numpy as np
import pandas as pd
import scipy.stats as st
import seaborn as sns

from matplotlib import rc
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Rectangle

# setup matplotlib
%matplotlib inline
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
sns.set_style('ticks')
sns.set_context('paper')
sns.set_style('ticks')
sns.set_context('paper')

# setup dict for specifying nice names for legends
optimizer_nice_names = {
    'constant': 'Constant',
    'adam': 'Adam',
    'momentum': 'Momentum',
    'rms': 'RMSProp',
    'sgd': 'SGD'}

# setup constant values for plots
X_LABEL_PAD = 5
Y_LABEL_PAD = 10

<a href=#top>Back to Top</a>

## MNIST<a name='mnist'/>

### Validation Phase<a name='mnist_validation'/>

In [ ]:
rdf = mnist_tools.load_data(['mnist_validation.json'])
rdf = rdf[rdf['success']]
for i in rdf.index:
    if rdf.at[i, 'optimizer'] == 'sgd':
        if float(rdf.at[i, 'momentum']) > 0:
            rdf.at[i, 'optimizer'] = 'momentum'
        else:
            rdf.at[i, 'momentum'] = np.nan

metric = mnist_tools.total_time_metric(50, 2500)
best = mnist_tools.get_best(
    mnist_tools.get_summary(rdf),
    metric,
    ['optimizer', 'momentum', 'rho'])
key = ('sgd',)
print('{0}: lr={1}, metric={2:.2f}, count={3}'.format(
    optimizer_nice_names[key[0]],
    best[key]['lr'],
    metric(best[key]),
    int(best[key]['count'][0])))
for momentum in np.sort(rdf['momentum'].unique()):
    key = ('momentum', momentum)
    if key not in best:
        continue
    print('{0} momentum={1}: lr={2}, metric={3:.2f}, count={4}'.format(
        optimizer_nice_names[key[0]],
        momentum,
        best[key]['lr'],
        metric(best[key]),
        int(best[key]['count'][0])))
for rho in np.sort(rdf['rho'].unique()):
    key = ('rms', rho)
    if key not in best:
        continue
    print('{0} rho={1}: lr={2}, metric={3:.2f}, count={4}'.format(
        optimizer_nice_names[key[0]],
        rho,
        best[key]['lr'],
        metric(best[key]),
        int(best[key]['count'][0])))
key = ('adam',)
print('{0} beta_1={1} beta_2={2}: lr={3}, metric={4:.2f}, count={5}'.format(
    optimizer_nice_names[key[0]],
    best[key]['beta_1'],
    best[key]['beta_2'],
    best[key]['lr'],
    metric(best[key]),
    int(best[key]['count'][0])))

In [ ]:
best = mnist_tools.get_best(
    mnist_tools.get_summary(rdf),
    metric,
    ['optimizer', 'lr', 'momentum', 'rho'])
summary = dict()
for k, v in best.items():
    if (k[0] == 'momentum') and (k[2] != 0.9):
        continue
    if (k[0] == 'rms') and (k[2] != 0.999):
        continue
    if k[0] not in summary:
        summary[k[0]] = dict()
    assert(k[1] not in summary[k[0]])
    summary[k[0]][k[1]] = v

max_phase_lengths = list()
for _, row in rdf.iterrows():
    for i, l in enumerate(row['phase_length']):
        if len(max_phase_lengths) == i:
            max_phase_lengths.append(0)
        max_phase_lengths[i] = max(max_phase_lengths[i], l)
data = dict()
for k in best.keys():
    data[k] = dict()
    for k2 in ['d1_count',
               'd2_count',
               'online_count',
               'as_count',
               'pi_count']:
        data[k][k2] = [np.zeros(i, dtype=int) for i in max_phase_lengths]
    for k2 in ['d1_avg',
               'd1_sec',
               'd1_min',
               'd1_max',
               'd2_avg',
               'd2_sec',
               'd2_min',
               'd2_max',
               'online_avg',
               'online_sec',
               'online_min',
               'online_max',
               'as_avg',
               'as_sec',
               'as_min',
               'as_max',
               'pi_avg',
               'pi_sec',
               'pi_min',
               'pi_max']:
        data[k][k2] = [np.zeros(i, dtype=float) for i in max_phase_lengths]
    for k2 in ['d1_final_avg',
               'd1_final_sec',
               'd2_final_avg',
               'd2_final_sec',
               'online_final_avg',
               'online_final_sec',
               'as_final_avg',
               'as_final_sec',
               'pi_final_avg',
               'pi_final_sec']:
        data[k][k2] = [0 for _ in range(len(max_phase_lengths))]
    data[k]['phase_length_count'] = [0 for i in max_phase_lengths]
    data[k]['phase_length_avg'] = [0 for i in max_phase_lengths]
    data[k]['phase_length_sec'] = [0 for i in max_phase_lengths]
    for i in range(len(max_phase_lengths)):
        data[k]['d1_min'][i] += 1
        data[k]['d2_min'][i] += 1
    kdf = rdf[rdf['optimizer'] == k[0]]
    kdf = kdf[kdf['lr'] == k[1]]
    if k[0] == 'momentum':
        kdf = kdf[kdf['momentum'] == k[2]]
    if k[0] == 'rms':
        kdf = kdf[kdf['rho'] == k[2]]
    for _, row in kdf.iterrows():
        j = 0
        for i, l in enumerate(row['phase_length']):
            delta = l - data[k]['phase_length_avg'][i]
            data[k]['phase_length_count'][i] += 1
            data[k]['phase_length_avg'][i] += delta / data[k]['phase_length_count'][i]
            data[k]['phase_length_sec'][i] += delta * (l - data[k]['phase_length_avg'][i])

            values = np.array(row['accuracies'])[j:j + l, 0]
            mask = np.where(np.array(np.invert(np.isnan(values)), dtype=int))[0]
            values = values[mask]
            delta = values - data[k]['d1_avg'][i][mask]
            data[k]['d1_count'][i][mask] += 1
            data[k]['d1_avg'][i][mask] += delta / data[k]['d1_count'][i][mask]
            data[k]['d1_sec'][i][mask] += delta * (values - data[k]['d1_avg'][i][mask])
            data[k]['d1_min'][i][mask] = np.minimum(data[k]['d1_min'][i][mask], values)
            data[k]['d1_max'][i][mask] = np.maximum(data[k]['d1_max'][i][mask], values)
            delta = values[-1] - data[k]['d1_final_avg'][i]
            data[k]['d1_final_avg'][i] += delta / data[k]['d1_count'][i][0]
            data[k]['d1_final_sec'][i] += delta * (values[-1] - data[k]['d1_final_avg'][i])

            values = np.array(row['accuracies'])[j:j + l, 1]
            mask = np.where(np.array(np.invert(np.isnan(values)), dtype=int))[0]
            values = values[mask]
            delta = values - data[k]['d2_avg'][i][mask]
            data[k]['d2_count'][i][mask] += 1
            data[k]['d2_avg'][i][mask] += delta / data[k]['d2_count'][i][mask]
            data[k]['d2_sec'][i][mask] += delta * (values - data[k]['d2_avg'][i][mask])
            data[k]['d2_min'][i][mask] = np.minimum(data[k]['d2_min'][i][mask], values)
            data[k]['d2_max'][i][mask] = np.maximum(data[k]['d2_max'][i][mask], values)
            delta = values[-1] - data[k]['d2_final_avg'][i]
            data[k]['d2_final_avg'][i] += delta / data[k]['d2_count'][i][0]
            data[k]['d2_final_sec'][i] += delta * (values[-1] - data[k]['d2_final_avg'][i])

            values = np.cumsum(np.array(row['correct'])[j:j + l]) / (np.arange(l) + 1)
            mask = np.where(np.array(np.invert(np.isnan(values)), dtype=int))[0]
            values = values[mask]
            delta = values - data[k]['online_avg'][i][mask]
            data[k]['online_count'][i][mask] += 1
            data[k]['online_avg'][i][mask] += delta / data[k]['online_count'][i][mask]
            data[k]['online_sec'][i][mask] += delta * (values - data[k]['online_avg'][i][mask])
            data[k]['online_min'][i][mask] = np.minimum(data[k]['online_min'][i][mask], values)
            data[k]['online_max'][i][mask] = np.maximum(data[k]['online_max'][i][mask], values)
            delta = values[-1] - data[k]['online_final_avg'][i]
            data[k]['online_final_avg'][i] += delta / data[k]['online_count'][i][0]
            data[k]['online_final_sec'][i] += delta * (values[-1] - data[k]['online_final_avg'][i])

            values = np.array(row['activation_similarity'])[j:j + l]
            mask = np.where(np.array(np.invert(np.isnan(values)), dtype=int))[0]
            values = values[mask]
            delta = values - data[k]['as_avg'][i][mask]
            data[k]['as_count'][i][mask] += 1
            data[k]['as_avg'][i][mask] += delta / data[k]['as_count'][i][mask]
            data[k]['as_sec'][i][mask] += delta * (values - data[k]['as_avg'][i][mask])
            data[k]['as_min'][i][mask] = np.minimum(data[k]['as_min'][i][mask], values)
            data[k]['as_max'][i][mask] = np.maximum(data[k]['as_max'][i][mask], values)
            delta = values[-1] - data[k]['as_final_avg'][i]
            data[k]['as_final_avg'][i] += delta / data[k]['as_count'][i][0]
            data[k]['as_final_sec'][i] += delta * (values[-1] - data[k]['as_final_avg'][i])

            values = np.array(row['pairwise_interference'])[j:j + l]
            mask = np.where(np.array(np.invert(np.isnan(values)), dtype=int))[0]
            values = values[mask]
            delta = values - data[k]['pi_avg'][i][mask]
            data[k]['pi_count'][i][mask] += 1
            data[k]['pi_avg'][i][mask] += delta / data[k]['pi_count'][i][mask]
            data[k]['pi_sec'][i][mask] += delta * (values - data[k]['pi_avg'][i][mask])
            data[k]['pi_min'][i][mask] = np.minimum(data[k]['pi_min'][i][mask], values)
            data[k]['pi_max'][i][mask] = np.maximum(data[k]['pi_max'][i][mask], values)
            delta = values[-1] - data[k]['pi_final_avg'][i]
            data[k]['pi_final_avg'][i] += delta / data[k]['pi_count'][i][0]
            data[k]['pi_final_sec'][i] += delta * (values[-1] - data[k]['pi_final_avg'][i])

            j += l
    data[k]['phase_length_sem'] = list()
    data[k]['d1_sem'] = list()
    data[k]['d2_sem'] = list()
    data[k]['online_sem'] = list()
    data[k]['as_sem'] = list()
    data[k]['pi_sem'] = list()
    data[k]['d1_final_sem'] = list()
    data[k]['d2_final_sem'] = list()
    data[k]['online_final_sem'] = list()
    data[k]['as_final_sem'] = list()
    data[k]['pi_final_sem'] = list()
    for i in range(len(max_phase_lengths)):
        data[k]['phase_length_sem'].append(np.sqrt(data[k]['phase_length_sec'][i]) / data[k]['phase_length_count'][i])
        data[k]['d1_sem'].append(np.nan_to_num(np.sqrt(data[k]['d1_sec'][i]) / data[k]['d1_count'][i]))
        data[k]['d2_sem'].append(np.nan_to_num(np.sqrt(data[k]['d2_sec'][i]) / data[k]['d2_count'][i]))
        data[k]['online_sem'].append(np.nan_to_num(np.sqrt(data[k]['online_sec'][i]) / data[k]['online_count'][i]))
        data[k]['as_sem'].append(np.nan_to_num(np.sqrt(data[k]['as_sec'][i]) / data[k]['as_count'][i]))
        data[k]['pi_sem'].append(np.nan_to_num(np.sqrt(data[k]['pi_sec'][i]) / data[k]['pi_count'][i]))
        data[k]['d1_final_sem'].append(np.nan_to_num(np.sqrt(data[k]['d1_final_sec'][i]) / data[k]['d1_count'][i][0]))
        data[k]['d2_final_sem'].append(np.nan_to_num(np.sqrt(data[k]['d2_final_sec'][i]) / data[k]['d2_count'][i][0]))
        data[k]['online_final_sem'].append(np.nan_to_num(np.sqrt(data[k]['online_final_sec'][i]) / data[k]['online_count'][i][0]))
        data[k]['as_final_sem'].append(np.nan_to_num(np.sqrt(data[k]['as_final_sec'][i]) / data[k]['as_count'][i][0]))
        data[k]['pi_final_sem'].append(np.nan_to_num(np.sqrt(data[k]['pi_final_sec'][i]) / data[k]['pi_count'][i][0]))

    del data[k]['phase_length_sec']
    del data[k]['d1_sec']
    del data[k]['d2_sec']
    del data[k]['online_sec']
    del data[k]['as_sec']
    del data[k]['pi_sec']
    del data[k]['d1_final_sec']
    del data[k]['d2_final_sec']
    del data[k]['online_final_sec']
    del data[k]['as_final_sec']
    del data[k]['pi_final_sec']

collected = {k[0]: dict() for k in data.keys()}
for k, v in data.items():
    if (k[0] == 'momentum') and (k[2] != 0.9):
        continue
    if (k[0] == 'rms') and (k[2] != 0.999):
        continue
    collected[k[0]][k[1]] = v
ordered = {k: dict() for k in collected.keys()}
for k, v in collected.items():
    ordered[k]['lr'] = sorted(v.keys())
    for k2 in ['phase_length',
               'd1_final',
               'd2_final',
               'online_final',
               'as_final',
               'pi_final']:
        ordered[k][k2 + '_avg'] = list()
        ordered[k][k2 + '_sem'] = list()
    for lr in ordered[k]['lr']:
        for k2 in ['phase_length',
                   'd1_final',
                   'd2_final',
                   'online_final',
                   'as_final',
                   'pi_final']:
            ordered[k][k2 + '_avg'].append(collected[k][lr][k2 + '_avg'])
            ordered[k][k2 + '_sem'].append(collected[k][lr][k2 + '_sem'])
    for k2 in ['phase_length','d1_final',
               'd2_final',
               'online_final',
               'as_final',
               'pi_final']:
        ordered[k][k2 + '_avg'] = np.array(ordered[k][k2 + '_avg'])
        ordered[k][k2 + '_sem'] = np.array(ordered[k][k2 + '_sem'])

In [ ]:
fig, axmat = plt.subplots(4, 2, figsize=(5.5, 5.5), dpi=300, sharex=True)
colors = sns.color_palette('colorblind', len(ordered.keys()))
xmax = 0
for j in range(4):
    for i, (k, v) in enumerate(sorted(ordered.items(), key=lambda x: x[0])):
        label = optimizer_nice_names[k] if j == 0 else None
        other_phase = (j + 1) % 2 + 1
        x = v['lr']
        y = v['d{}_final_avg'.format(other_phase)][:, j]
        yerr = v['d{}_final_sem'.format(other_phase)][:, j]
        axmat[j, 0].plot(
            x,
            y,
            '-o',
            linestyle='--',
            linewidth=1,
            markersize=2,
            color=colors[i])
        axmat[j, 0].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

        y = v['online_final_avg'][:, j]
        yerr = v['online_final_sem'][:, j]
        axmat[j, 0].plot(
            x,
            y,
            '-o',
            label=label,
            linewidth=1,
            markersize=2,
            color=colors[i])
        axmat[j, 0].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

        y = v['phase_length_avg'][:, j]
        yerr = v['phase_length_avg'][:, j]
        axmat[j, 1].plot(
            x,
            y,
            '-o',
            linewidth=1,
            markersize=2,
            color=colors[i])
        axmat[j, 1].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

for j in range(4):
    axmat[j, 0].set_ylim(-0.05, 1.05)
    axmat[j, 0].set_yticks([0.0, 0.5, 1.0])
    axmat[j, 0].set_ylabel('Phase {}'.format(j + 1), labelpad=Y_LABEL_PAD)
    axmat[j, 1].set_ylim(- 225, 4500)
    axmat[j, 1].set_yticks([0, 2000, 4000])
axmat[0, 0].set_title('Final Accuracy')
axmat[0, 1].set_title('Steps in Phase')
axmat[-1, 0].set_xscale('log', basex=2)
axmat[-1, 0].set_xticks([2 ** (-18), 2 ** (-13), 2 ** (-8), 2 ** (-3)])
axmat[-1, 0].set_xlim(2 ** (-18), 2 ** (-3))
axmat[-1, 0].set_xlabel('Step-size', labelpad=X_LABEL_PAD)
axmat[-1, 1].set_xlabel('Step-size', labelpad=X_LABEL_PAD)
fig.tight_layout()
fig.subplots_adjust(hspace=0.15, wspace=0.3, right=0.775)
pos = axmat[0, 0].get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('mnist_accuracies_and_speed_step-size.pdf')

In [ ]:
learning_rates = [0.125,
                  0.0625,
                  0.03125,
                  0.015625,
                  0.0078125,
                  0.00390625,
                  0.001953125,
                  0.0009765625,
                  0.00048828125,
                  0.000244140625,
                  0.0001220703125,
                  6.103515625e-05,
                  3.0517578125e-05,
                  1.52587890625e-05,
                  7.62939453125e-06,
                  3.814697265625e-06]

retention = dict()
relearning = dict()
for _, row in rdf.iterrows():
    optimizer = row['optimizer']
    if optimizer == 'constant':
        continue
    if (optimizer == 'momentum') and (row['momentum'] != 0.9):
        continue
    if (optimizer == 'rms') and (row['rho'] != 0.999):
        continue
    lr = row['lr']
    if optimizer not in retention:
        assert(optimizer not in relearning)
        for dict_ in [retention, relearning]:
            dict_[optimizer] = collections.OrderedDict()
            for value in learning_rates:
                dict_[optimizer][value] = list()
    retention[optimizer][lr].append(row['accuracies'][sum(row['phase_length'][:2]) - 1][0])
    relearning[optimizer][lr].append(row['phase_length'][0] / row['phase_length'][2])

colors = sns.color_palette(n_colors=len(retention.keys()))
fig, (ax1, ax2) = plt.subplots(1, 2, dpi=300, figsize=(5.5, 1.85))
sort_key = lambda x: x[0]
for i, optimizer in enumerate(sorted(retention.keys())):
    x = list(retention[optimizer].keys())
    y1 = list()
    y1err = list()
    y2 = list()
    y2err = list()
    for lr in x:
        for values, y, yerr in [(retention[optimizer][lr], y1, y1err),
                                (relearning[optimizer][lr], y2, y2err)]:
            y.append(np.mean(values))
            yerr.append(np.std(values) / np.sqrt(len(values)))
    x = np.array(x)
    y1 = np.array(y1)
    y1err = np.array(y1err)
    y2 = np.array(y2)
    y2err = np.array(y2err)

    ax1.plot(
        x,
        y1,
        '-o',
        label=optimizer_nice_names[optimizer],
        markersize=2,
        linewidth=1,
        color=colors[i])
    ax1.fill_between(
        x,
        y1 - y1err,
        y1 + y1err,
        alpha=0.3,
        color=colors[i])

    ax2.plot(
        x,
        y2,
        '-o',
        linewidth=1,
        markersize=2,
        color=colors[i])
    ax2.fill_between(
        x,
        y2 - y2err,
        y2 + y2err,
        alpha=0.3,
        color=colors[i])

for ax in [ax1, ax2]:
    ax.set_xscale('log', basex=2)
    ax.set_xlim(2 ** (-18), 2 ** (-3))
    ax.set_xticks([2 ** (-15), 2 ** (-10), 2 ** (-5)])
    ax.set_xlabel('Step-size', labelpad=X_LABEL_PAD)
ax1.set_ylabel('Retention', labelpad=Y_LABEL_PAD)
ax1.set_ylim(-0.05, 1.05)
ax1.set_yticks([0.0, 0.5, 1.0])
ax2.set_ylabel('Relearning', labelpad=Y_LABEL_PAD)
ax2.set_ylim(-0.25, 4.75)
ax2.set_yticks([0, 2, 4])
fig.tight_layout()
fig.subplots_adjust(hspace=0.15, wspace=0.4, right=0.765)
pos = ax1.get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('mnist_interference_step-size.pdf')

In [ ]:
threshold = 125
fig, axmat = plt.subplots(2, 4, sharex=True, dpi=300, figsize=(5.5, 3))
colors = sns.color_palette('colorblind', len(data.keys()))
xmax = 0
for j in range(4):
    for i, (k, v) in enumerate(sorted(ordered.items(), key=lambda x: x[0])):
        x = v['lr']
        label = optimizer_nice_names[k] if j == 0 else None
        y = v['as_final_avg'][:, j]
        yerr = v['as_final_sem'][:, j]
        axmat[0, j].plot(
            x,
            y,
            '-o',
            label=label,
            linewidth=1,
            markersize=2,
            color=colors[i])
        axmat[0, j].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

        y = v['pi_final_avg'][:, j]
        yerr = v['pi_final_sem'][:, j]
        axmat[1, j].plot(
            x,
            y,
            '-o',
            linewidth=1,
            markersize=2,
            color=colors[i])
        axmat[1, j].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

axmat[0, 0].set_ylabel('Final Activation\nOverlap', labelpad=Y_LABEL_PAD)
axmat[1, 0].set_ylabel('Final Pairwise\nInterference', labelpad=Y_LABEL_PAD)
for j in range(4):
    axmat[0, j].set_title('Phase {}'.format(j + 1))
    axmat[0, j].set_ylim(- 100, 1400)
    axmat[0, j].set_yticks([0, 500, 1000])
    axmat[1, j].set_ylim(- 0.5, 0.6)
    axmat[1, j].set_yticks([- 0.3, 0, 0.3])
    for i in range(2):
        axmat[i, j].set_xscale('log', basex=2)
        axmat[i, j].set_xlim(2 ** (-18), 2 ** (-3))
        if j != 0:
            axmat[i, j].tick_params(axis='y', which='both', left=False)
            axmat[i, j].set_yticklabels([])
    axmat[1, j].set_xticks([2 ** (-15), 2 ** (-10), 2 ** (-5)])
    axmat[1, j].set_xlabel('Step-size', labelpad=X_LABEL_PAD)
fig.align_ylabels(axmat[:, 0])
fig.tight_layout()
fig.subplots_adjust(hspace=0.0, wspace=0.0, right=0.775)
pos = axmat[0, 0].get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('mnist_additional_interference_step-size.pdf')

<a href=#top>Back to Top</a>

### Testing Phase<a name='mnist_testing'/>

In [ ]:
rdf = mnist_tools.load_data(['mnist_test.json'])
rdf = rdf[rdf['success']]
for i in rdf.index:
    if rdf.at[i, 'optimizer'] == 'sgd':
        if float(rdf.at[i, 'momentum']) > 0:
            rdf.at[i, 'optimizer'] = 'momentum'
        else:
            rdf.at[i, 'momentum'] = None
best = mnist_tools.get_best(
    mnist_tools.get_summary(rdf),
    mnist_tools.total_time_metric(50, 2500),
    ['optimizer', 'momentum', 'rho'])

max_phase_lengths = list()
for _, row in rdf.iterrows():
    for i, l in enumerate(row['phase_length']):
        if len(max_phase_lengths) == i:
            max_phase_lengths.append(0)
        max_phase_lengths[i] = max(max_phase_lengths[i], l)
data = dict()
for k in best.keys():
    data[k] = dict()
    for k2 in ['d1_count',
               'd2_count',
               'online_count',
               'as_count',
               'pi_count']:
        data[k][k2] = [np.zeros(i, dtype=int) for i in max_phase_lengths]
    for k2 in ['d1_avg',
               'd1_sec',
               'd1_min',
               'd1_max',
               'd2_avg',
               'd2_sec',
               'd2_min',
               'd2_max',
               'online_avg',
               'online_sec',
               'online_min',
               'online_max',
               'as_avg',
               'as_sec',
               'as_min',
               'as_max',
               'pi_avg',
               'pi_sec',
               'pi_min',
               'pi_max']:
        data[k][k2] = [np.zeros(i, dtype=float) for i in max_phase_lengths]
    data[k]['phase_lengths'] = [list() for i in max_phase_lengths]
    for i in range(len(max_phase_lengths)):
        data[k]['d1_min'][i] += 1
        data[k]['d2_min'][i] += 1
    kdf = rdf[rdf['optimizer'] == k[0]]
    if k[0] == 'momentum':
        kdf = kdf[kdf['momentum'] == k[1]]
    if k[0] == 'rms':
        kdf = kdf[kdf['rho'] == k[1]]
    for _, row in kdf.iterrows():
        j = 0
        for i, l in enumerate(row['phase_length']):
            data[k]['phase_lengths'][i].append(l)

            values = np.array(row['accuracies'])[j:j + l, 0]
            mask = np.where(np.array(np.invert(np.isnan(values)), dtype=int))[0]
            values = values[mask]
            delta = values - data[k]['d1_avg'][i][mask]
            data[k]['d1_count'][i][mask] += 1
            data[k]['d1_avg'][i][mask] += delta / data[k]['d1_count'][i][mask]
            data[k]['d1_sec'][i][mask] += delta * (values - data[k]['d1_avg'][i][mask])
            data[k]['d1_min'][i][mask] = np.minimum(data[k]['d1_min'][i][mask], values)
            data[k]['d1_max'][i][mask] = np.maximum(data[k]['d1_max'][i][mask], values)

            values = np.array(row['accuracies'])[j:j + l, 1]
            mask = np.where(np.array(np.invert(np.isnan(values)), dtype=int))[0]
            values = values[mask]
            delta = values - data[k]['d2_avg'][i][mask]
            data[k]['d2_count'][i][mask] += 1
            data[k]['d2_avg'][i][mask] += delta / data[k]['d2_count'][i][mask]
            data[k]['d2_sec'][i][mask] += delta * (values - data[k]['d2_avg'][i][mask])
            data[k]['d2_min'][i][mask] = np.minimum(data[k]['d2_min'][i][mask], values)
            data[k]['d2_max'][i][mask] = np.maximum(data[k]['d2_max'][i][mask], values)

            values = np.cumsum(np.array(row['correct'])[j:j + l]) / (np.arange(l) + 1)
            mask = np.where(np.array(np.invert(np.isnan(values)), dtype=int))[0]
            values = values[mask]
            delta = values - data[k]['online_avg'][i][mask]
            data[k]['online_count'][i][mask] += 1
            data[k]['online_avg'][i][mask] += delta / data[k]['online_count'][i][mask]
            data[k]['online_sec'][i][mask] += delta * (values - data[k]['online_avg'][i][mask])
            data[k]['online_min'][i][mask] = np.minimum(data[k]['online_min'][i][mask], values)
            data[k]['online_max'][i][mask] = np.maximum(data[k]['online_max'][i][mask], values)

            values = np.array(row['activation_similarity'])[j:j + l]
            mask = np.where(np.array(np.invert(np.isnan(values)), dtype=int))[0]
            values = values[mask]
            delta = values - data[k]['as_avg'][i][mask]
            data[k]['as_count'][i][mask] += 1
            data[k]['as_avg'][i][mask] += delta / data[k]['as_count'][i][mask]
            data[k]['as_sec'][i][mask] += delta * (values - data[k]['as_avg'][i][mask])
            data[k]['as_min'][i][mask] = np.minimum(data[k]['as_min'][i][mask], values)
            data[k]['as_max'][i][mask] = np.maximum(data[k]['as_max'][i][mask], values)

            values = np.array(row['pairwise_interference'])[j:j + l]
            mask = np.where(np.array(np.invert(np.isnan(values)), dtype=int))[0]
            values = values[mask]
            delta = values - data[k]['pi_avg'][i][mask]
            data[k]['pi_count'][i][mask] += 1
            data[k]['pi_avg'][i][mask] += delta / data[k]['pi_count'][i][mask]
            data[k]['pi_sec'][i][mask] += delta * (values - data[k]['pi_avg'][i][mask])
            data[k]['pi_min'][i][mask] = np.minimum(data[k]['pi_min'][i][mask], values)
            data[k]['pi_max'][i][mask] = np.maximum(data[k]['pi_max'][i][mask], values)

            j += l
    data[k]['d1_sem'] = list()
    data[k]['d2_sem'] = list()
    data[k]['online_sem'] = list()
    data[k]['as_sem'] = list()
    data[k]['pi_sem'] = list()
    for i in range(len(max_phase_lengths)):
        data[k]['d1_sem'].append(np.nan_to_num(np.sqrt(data[k]['d1_sec'][i]) / data[k]['d1_count'][i]))
        data[k]['d2_sem'].append(np.nan_to_num(np.sqrt(data[k]['d2_sec'][i]) / data[k]['d2_count'][i]))
        data[k]['online_sem'].append(np.nan_to_num(np.sqrt(data[k]['online_sec'][i]) / data[k]['online_count'][i]))
        data[k]['as_sem'].append(np.nan_to_num(np.sqrt(data[k]['as_sec'][i]) / data[k]['as_count'][i]))
        data[k]['pi_sem'].append(np.nan_to_num(np.sqrt(data[k]['pi_sec'][i]) / data[k]['pi_count'][i]))
    del data[k]['d1_sec']
    del data[k]['d2_sec']
    del data[k]['online_sec']
    del data[k]['as_sec']
    del data[k]['pi_sec']

In [ ]:
plot_data = {k: v for k, v in data.items() if (len(k) == 1) or ((k[0] == 'momentum') and (k[1] == 0.9)) or ((k[0] == 'rms') and (k[1] == 0.999))}
threshold = 125
fig, axarr = plt.subplots(4, 1, figsize=(5.5, 5.5), dpi=300, sharex=True)
colors = sns.color_palette('colorblind', len(plot_data.keys()))
xmax = 0
for j in range(len(axarr)):
    ax = axarr[j]
    for i, (k, v) in enumerate(sorted(plot_data.items(), key=lambda x: x[0])):
        other_phase = (j + 1) % 2 + 1
        x = np.where(v['d{}_count'.format(other_phase)][j] >= threshold)[0]
        xmax = max(xmax, max(x))
        y = v['d{}_avg'.format(other_phase)][j][x]
        yerr = v['d{}_sem'.format(other_phase)][j][x]
        ax.fill_between(x, y - yerr, y + yerr, color=colors[i], alpha=0.1)
        ax.plot(x, y, color=colors[i], linestyle='--', linewidth=1)

        label = optimizer_nice_names[k[0]] if j == 0 else None
        x = np.where(v['online_count'][j] >= threshold)[0]
        xmax = max(xmax, max(x))
        y = v['online_avg'][j][x]
        yerr = v['online_sem'][j][x]
        ax.fill_between(x, y - yerr, y + yerr, color=colors[i], alpha=0.1)
        ax.plot(x, y, label=label, color=colors[i], linestyle='-', linewidth=1)

for j in range(len(axarr)):
    ax = axarr[j]
    ax.set_ylim(-0.05, 1.05)
    ax.set_yticks([0.0, 0.5, 1.0])
    ax.set_xlim((0, xmax * 1.05))
    ax.set_ylabel('Phase {}'.format(j + 1), labelpad=Y_LABEL_PAD)
axarr[0].set_title('Accuracy')
axarr[-1].set_xticks([0, 50, 100, 150, 200, 250])
axarr[-1].set_xlim(0, 250)
axarr[-1].set_xlabel('Steps', labelpad=X_LABEL_PAD)
fig.tight_layout()
fig.subplots_adjust(hspace=0.15, right=0.775)
pos = axarr[0].get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('mnist_accuracies.pdf')

In [ ]:
phase_lengths = [{optimizer: list() for optimizer in rdf['optimizer'].unique()} for _ in range(4)]
for _, row in rdf.iterrows():
    if (row['optimizer'] == 'momentum') and (row['momentum'] != 0.9):
        continue
    if (row['optimizer'] == 'rms') and (row['rho'] != 0.999):
        continue
    for i in range(4):
        phase_lengths[i][row['optimizer']].append(row['phase_length'][i])

text = ''
text += '|-----------|------------------|------------------|------------------|------------------|\n'
text += '| Optimizer | Steps in Phase 1 | Steps in Phase 2 | Steps in Phase 3 | Steps in Phase 4 |\n'
text += '|-----------|------------------|------------------|------------------|------------------|\n'
for optimizer in sorted(list(phase_lengths[0].keys())):
    text += '| {0:>9} |   {1:>6.2f}+-{2:<4.2f}   |   {3:>6.2f}+-{4:<4.2f}   |   {5:>6.2f}+-{6:<4.2f}   |   {7:>6.2f}+-{8:<4.2f}   |\n'.format(
        optimizer_nice_names[optimizer],
        np.mean(phase_lengths[0][optimizer]), np.std(phase_lengths[0][optimizer]) / np.sqrt(len(phase_lengths[0][optimizer])),
        np.mean(phase_lengths[1][optimizer]), np.std(phase_lengths[1][optimizer]) / np.sqrt(len(phase_lengths[1][optimizer])),
        np.mean(phase_lengths[2][optimizer]), np.std(phase_lengths[2][optimizer]) / np.sqrt(len(phase_lengths[2][optimizer])),
        np.mean(phase_lengths[3][optimizer]), np.std(phase_lengths[3][optimizer]) / np.sqrt(len(phase_lengths[3][optimizer])))
text += '|-----------|------------------|------------------|------------------|------------------|'
with open('mnist_speed.txt', 'w') as outfile:
    print(text, file=outfile)
print(text)

In [ ]:
accuracies = {name: [[list() for j in range(2)] for i in range(4)] for name in list(set(rdf['optimizer'].unique()) - {'constant'})}
for _, row in rdf.iterrows():
    if row['optimizer'] == 'constant':
        continue
    if (row['optimizer'] == 'momentum') and (row['momentum'] != 0.9):
        continue
    if (row['optimizer'] == 'rms') and (row['rho'] != 0.999):
        continue
    for i in range(4):
        for j in range(2):
            accuracies[row['optimizer']][i][j].append(row['accuracies'][sum(row['phase_length'][:i + 1]) - 1][j])

text = ''
text += '|-----------|-------|-------------------|-------------------|\n'
text += '| Optimizer | Phase | Accuracy on 1 + 2 | Accuracy on 3 + 4 |\n'
text += '|-----------|-------|-------------------|-------------------|\n'
for optimizer in sorted(list(accuracies.keys())):
    for phase in range(4):
        if (optimizer != 'adam') and (phase == 0):
            text += '|-----------|-------|-------------------|-------------------|\n'
        text += '| {0:>9} |   {1}   |  {2:>6.4f}+-{3:<6.4f}   |  {4:>6.4f}+-{5:<6.4f}   |\n'.format(
            optimizer_nice_names[optimizer] if phase == 1 else '',
            phase,
            np.mean(accuracies[optimizer][phase][0]), np.std(accuracies[optimizer][phase][0]) / np.sqrt(len(accuracies[optimizer][phase][0])),
            np.mean(accuracies[optimizer][phase][1]), np.std(accuracies[optimizer][phase][1]) / np.sqrt(len(accuracies[optimizer][phase][1])))
text += '|-----------|-------|-------------------|-------------------|\n'
with open('mnist_retention.txt', 'w') as outfile:
    print(text, file=outfile)
print(text)

In [ ]:
phase_lengths = [{optimizer: list() for optimizer in rdf['optimizer'].unique()} for _ in range(4)]
for _, row in rdf.iterrows():
    if (row['optimizer'] == 'momentum') and (row['momentum'] != 0.9):
        continue
    if (row['optimizer'] == 'rms') and (row['rho'] != 0.999):
        continue
    for i in range(4):
        phase_lengths[i][row['optimizer']].append(row['phase_length'][i])
boxplot_data = {'optimizer': [], 'phase': [], 'phase_length': []}
for optimizer in sorted(list(phase_lengths[2].keys())):
    for i in range(len(phase_lengths[0][optimizer])):
        first_phase_length = phase_lengths[0][optimizer][i]
        third_phase_length = phase_lengths[2][optimizer][i]
        boxplot_data['optimizer'].append(optimizer_nice_names[optimizer])
        boxplot_data['phase'].append('Phase 1')
        boxplot_data['phase_length'].append(first_phase_length)
        boxplot_data['optimizer'].append(optimizer_nice_names[optimizer])
        boxplot_data['phase'].append('Phase 3')
        boxplot_data['phase_length'].append(third_phase_length)

fig, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=300)
ax.set_xlabel('Phase Length', labelpad=X_LABEL_PAD)
sns.violinplot(x='phase_length', y='optimizer', hue='phase', data=boxplot_data, cut=0, split=True, inner=None, ax=ax, legend=None)
ax.set_xlim(0, 400)
ax.set_xticks([0, 100, 200, 300, 400])
sns.despine(offset=10, trim=True)
fig.tight_layout()
pos = ax.get_position()
lgd = plt.legend(bbox_to_anchor=(1, pos.y1), loc='upper left')
fig.savefig('mnist_relearning.pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
ratios = {optimizer: list() for optimizer in rdf['optimizer'].unique()}
for _, row in rdf.iterrows():
    if (row['optimizer'] == 'momentum') and (row['momentum'] != 0.9):
        continue
    if (row['optimizer'] == 'rms') and (row['rho'] != 0.999):
        continue
    ratios[row['optimizer']].append(row['phase_length'][0] / row['phase_length'][2])

text = ''
text += '|-----------|------------|\n'
text += '| Optimizer | Relearning |\n'
text += '|---------- |------------|\n'
for optimizer in sorted(list(ratios.keys())):
    text += '| {0:>9} | {1:>4.2f}+-{2:<4.2f} |\n'.format(
        optimizer_nice_names[optimizer],
        np.mean(ratios[optimizer]),
        np.std(ratios[optimizer]) / np.sqrt(len(ratios[optimizer])))
text += '|---------- |------------|\n'
with open('mnist_relearning.txt', 'w') as outfile:
    print(text, file=outfile)
print(text)

In [ ]:
plot_data = {k: v for k, v in data.items() if (len(k) == 1) or ((k[0] == 'momentum') and (k[1] == 0.9)) or ((k[0] == 'rms') and (k[1] == 0.999))}
threshold = 125
fig, axmat = plt.subplots(2, 4, sharex=True, dpi=300, figsize=(6, 3.5))
colors = sns.color_palette('colorblind', len(data.keys()))
xmax = 0
for j in range(4):
    for i, (k, v) in enumerate(sorted(plot_data.items(), key=lambda x: x[0])):
        x = np.where(v['as_count'][j] >= threshold)[0]
        xmax = max(xmax, max(x))
        y = v['as_avg'][j][x]
        yerr = v['as_sem'][j][x]
        label = optimizer_nice_names[k[0]] if j == 0 else None
        axmat[0, j].plot(
            x,
            y,
            label=label,
            linewidth=1,
            color=colors[i])
        axmat[0, j].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

        x = np.where(v['pi_count'][j] >= threshold)[0]
        xmax = max(xmax, max(x))
        y = - v['pi_avg'][j][x]
        yerr = v['pi_sem'][j][x]
        axmat[1, j].plot(
            x,
            y,
            linewidth=1,
            color=colors[i])
        axmat[1, j].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

axmat[0, 0].set_ylabel('Activation Overlap', labelpad=Y_LABEL_PAD)
axmat[1, 0].set_ylabel('Pairwise Interference', labelpad=Y_LABEL_PAD)
for i in range(4):
    axmat[0, i].set_title('Phase {}'.format(i + 1))
    axmat[1, i].set_xticks([0, 80, 160])
    axmat[1, i].set_xlabel('Steps', labelpad=X_LABEL_PAD)
    axmat[0, i].set_ylim(- 25, 250)
    axmat[0, i].set_yticks([0, 100, 200])
    axmat[1, i].set_ylim(- 0.175, 0.025)
    axmat[1, i].set_yticks([-0.15, -0.1, -0.05, 0.0])
    for j in range(2):
        axmat[j, i].set_xlim(0, xmax * 1.05)
        if i != 0:
            axmat[j, i].tick_params(axis='y', which='both', left=False)
            axmat[j, i].set_yticklabels([])
fig.align_ylabels(axmat[:, 0])
fig.tight_layout()
fig.subplots_adjust(hspace=0.0, wspace=0.0, right=0.775)
pos = axmat[0, 0].get_position()
fig.legend(bbox_to_anchor=(0.9825, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right', frameon=False)
fig.savefig('mnist_additional_interference.pdf')

In [ ]:
optimizers = sorted(list(rdf['optimizer'].unique()))
retention = {optimizer: list() for optimizer in optimizers}
relearning = copy.deepcopy(retention)
for _, row in rdf.iterrows():
    optimizer = row['optimizer']
    if (optimizer == 'momentum') and (row['momentum'] != 0.9):
        continue
    if (optimizer == 'rms') and (row['rho'] != 0.999):
        continue
    retention[optimizer].append(row['accuracies'][sum(row['phase_length'][:2]) - 1][0])
    relearning[optimizer].append(row['phase_length'][0] / row['phase_length'][2])

labels = list()
retention_y = list()
retention_yerr = list()
relearning_y = list()
relearning_yerr = list()
for optimizer in optimizers:
    labels.append(optimizer_nice_names[optimizer])
    retention_y.append(np.mean(retention[optimizer]))
    retention_yerr.append(np.std(retention[optimizer]) / np.sqrt(len(retention[optimizer])))
    relearning_y.append(np.mean(relearning[optimizer]))
    relearning_yerr.append(np.std(relearning[optimizer]) / np.sqrt(len(relearning[optimizer])))
x = np.arange(len(labels))

fig, ax1 = plt.subplots(1, 1, dpi=300, figsize=(4, 2))
ax2 = ax1.twinx()
colors = sns.color_palette('colorblind', 2)
width = 0.35
ax1.bar(x - width / 2,
        retention_y,
        width,
        yerr=retention_yerr,
        label='Retention',
        color=colors[0])
ax2.bar(x + width / 2,
        relearning_y,
        width,
        yerr=relearning_yerr,
        label='Relearning',
        color=colors[1])

ax1.set_xticks(x)
ax1.set_xticklabels(labels)
ax1.set_yticks([0, 0.1, 0.2, 0.3])
ax1.set_ylim(0, 0.3)
ax1.set_ylabel('Retention (left/blue)', labelpad=10)
ax2.set_yticks([0, 1, 2, 3, 4])
ax2.set_ylim(0, 4)
ax2.set_ylabel('Relearning (right/yellow)', labelpad=10)
fig.tight_layout()
fig.savefig('mnist_retention_and_relearning.pdf')

In [ ]:
accuracy = {optimizer: dict() for optimizer in ['momentum', 'rms']}
retention = copy.deepcopy(accuracy)
relearning = copy.deepcopy(accuracy)
for _, row in rdf.iterrows():
    optimizer = row['optimizer']
    if optimizer == 'momentum':
        gamma = row['momentum']
    elif optimizer == 'rms':
        gamma = row['rho']
    else:
        continue
    if gamma not in accuracy[optimizer]:
        assert(gamma not in retention[optimizer])
        assert(gamma not in relearning[optimizer])
        accuracy[optimizer][gamma] = list()
        retention[optimizer][gamma] = list()
        relearning[optimizer][gamma] = list()
    accuracy[optimizer][gamma].append(row['accuracies'][sum(row['phase_length'][:1]) - 1][0])
    retention[optimizer][gamma].append(row['accuracies'][sum(row['phase_length'][:2]) - 1][0])
    relearning[optimizer][gamma].append(row['phase_length'][0] / row['phase_length'][2])

text = ''
text += '|-----------|--------|----------------|----------------|\n'
text += '| Optimizer | Gamma  |   Retention    |   Relearning   |\n'
text += '|-----------|--------|----------------|----------------|\n'
for optimizer in sorted(list(accuracy.keys())):
    for i, gamma in enumerate(sorted(accuracy[optimizer].keys())):
        text += '| {0:>9} | {1}  | {2:>6.4f}+-{3:<6.4f} | {4:>6.4f}+-{5:<6.4f} |\n'.format(
            optimizer_nice_names[optimizer] if i == 1 else '',
            str(gamma).ljust(5),
            np.mean(retention[optimizer][gamma]),
            np.std(retention[optimizer][gamma]) / np.sqrt(len(retention[optimizer][gamma])),
            np.mean(relearning[optimizer][gamma]),
            np.std(relearning[optimizer][gamma]) / np.sqrt(len(relearning[optimizer][gamma])))
    text += '|-----------|--------|----------------|----------------|\n'
with open('mnist_interference_momentum_and_rms.txt', 'w') as outfile:
    print(text, file=outfile)
print(text)

In [ ]:
optimizers = ['momentum', 'rms']
retention = {optimizer: dict() for optimizer in optimizers}
relearning = copy.deepcopy(retention)
for _, row in rdf.iterrows():
    optimizer = row['optimizer']
    if optimizer == 'momentum':
        gamma = row['momentum']
    elif optimizer == 'rms':
        gamma = row['rho']
    else:
        continue
    if gamma not in retention[optimizer]:
        assert(gamma not in relearning[optimizer])
        retention[optimizer][gamma] = list()
        relearning[optimizer][gamma] = list()
    retention[optimizer][gamma].append(row['accuracies'][sum(row['phase_length'][:2]) - 1][0])
    relearning[optimizer][gamma].append(row['phase_length'][0] / row['phase_length'][2])

fig, axarr1 = plt.subplots(1, 2, dpi=300, figsize=(4.5, 2), gridspec_kw={'width_ratios': [3, 4]})
axarr2 = [ax.twinx() for ax in axarr1]
colors = sns.color_palette('colorblind', 2)
width = 0.35

x1 = np.arange(len(retention['momentum'].keys()))
y = [np.mean(v) for k, v in sorted(retention['momentum'].items(), key=lambda x: x[0])] 
yerr = [np.std(v) / np.sqrt(len(v)) for k, v in sorted(retention['momentum'].items(), key=lambda x: x[0])] 
axarr1[0].bar(x1 - width / 2,
        y,
        width,
        yerr=yerr,
        label='Retention',
        color=colors[0])
y = [np.mean(v) for k, v in sorted(relearning['momentum'].items(), key=lambda x: x[0])] 
yerr = [np.std(v) / np.sqrt(len(v)) for k, v in sorted(relearning['momentum'].items(), key=lambda x: x[0])]
axarr2[0].bar(x1 + width / 2,
        y,
        width,
        yerr=yerr,
        label='Relearning',
        color=colors[1])

x2 = np.arange(len(retention['rms'].keys()))
y = [np.mean(v) for k, v in sorted(retention['rms'].items(), key=lambda x: x[0])] 
yerr = [np.std(v) / np.sqrt(len(v)) for k, v in sorted(retention['rms'].items(), key=lambda x: x[0])]
axarr1[1].bar(x2 - width / 2,
        y,
        width,
        yerr=yerr,
        label='Retention',
        color=colors[0])
y = [np.mean(v) for k, v in sorted(relearning['rms'].items(), key=lambda x: x[0])] 
yerr = [np.std(v) / np.sqrt(len(v)) for k, v in sorted(relearning['rms'].items(), key=lambda x: x[0])]
axarr2[1].bar(x2 + width / 2,
        y,
        width,
        yerr=yerr,
        label='Relearning',
        color=colors[1])

axarr1[0].set_title('SGD with Momentum', pad=5)
axarr1[1].set_title('RMSProp', pad=5)
axarr1[0].set_xlabel('Momentum', labelpad=5)
axarr1[1].set_xlabel('Moving Average Coefficient', labelpad=5)
axarr1[0].set_yticks([0, 0.1, 0.2, 0.3])
axarr1[1].set_yticks([])
axarr2[0].set_yticks([])
axarr2[1].set_yticks([0, 1, 2, 3, 4])
for ax in axarr1:
    ax.set_ylim(0, 0.3)
for ax in axarr2:
    ax.set_ylim(0, 4)
axarr1[0].set_ylabel('Retention (left/blue)', labelpad=10)
axarr2[1].set_ylabel('Relearning (right/yellow)', labelpad=10)
axarr1[0].set_xticks(x1)
axarr1[0].set_xticklabels(sorted(list(retention['momentum'].keys())))
axarr1[1].set_xticks(x2)
axarr1[1].set_xticklabels(sorted(list(retention['rms'].keys())))

fig.tight_layout()
fig.subplots_adjust(wspace=0)
fig.savefig('mnist_interference_momentum_and_rms.pdf')

In [ ]:
plot_data = {
    r'$\beta={}$'.format(k[1]) : v
    for k, v in data.items()
    if k[0] == 'momentum'}
threshold = 125
fig, axmat = plt.subplots(3, 4, sharex=True, dpi=300, figsize=(7, 4))
colors = sns.color_palette('colorblind', len(data.keys()))
xmax = 0
for j in range(4):
    for i, (k, v) in enumerate(sorted(plot_data.items(), key=lambda x: x[0])):
        other_phase = (j + 1) % 2 + 1
        x = np.where(v['d{}_count'.format(other_phase)][j] >= threshold)[0]
        xmax = max(xmax, max(x))
        y = v['d{}_avg'.format(other_phase)][j][x]
        yerr = v['d{}_sem'.format(other_phase)][j][x]
        label = k if j == 0 else None
        axmat[0, j].plot(
            x,
            y,
            linestyle='--',
            linewidth=1,
            color=colors[i])
        axmat[0, j].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

        x = np.where(v['online_count'][j] >= threshold)[0]
        xmax = max(xmax, max(x))
        y = v['online_avg'][j][x]
        yerr = v['online_sem'][j][x]
        axmat[0, j].plot(
            x,
            y,
            label=label,
            linewidth=1,
            color=colors[i])
        axmat[0, j].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

        x = np.where(v['as_count'][j] >= threshold)[0]
        xmax = max(xmax, max(x))
        y = v['as_avg'][j][x]
        yerr = v['as_sem'][j][x]
        axmat[1, j].plot(
            x,
            y,
            linewidth=1,
            color=colors[i])
        axmat[1, j].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

        x = np.where(v['pi_count'][j] >= threshold)[0]
        xmax = max(xmax, max(x))
        y = - v['pi_avg'][j][x]
        yerr = v['pi_sem'][j][x]
        axmat[2, j].plot(
            x,
            y,
            linewidth=1,
            color=colors[i])
        axmat[2, j].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

axmat[0, 0].set_ylabel('Accuracy', labelpad=Y_LABEL_PAD)
axmat[1, 0].set_ylabel('Activation\nOverlap', labelpad=Y_LABEL_PAD)
axmat[2, 0].set_ylabel('Pairwise\nInterference', labelpad=Y_LABEL_PAD)
for i in range(4):
    axmat[0, i].set_title('Phase {}'.format(i + 1))
    axmat[0, i].set_ylim(-0.05, 1.05)
    axmat[0, i].set_yticks([0.0, 0.5, 1.0])
    axmat[1, i].set_ylim(- 25, 375)
    axmat[1, i].set_yticks([0, 100, 200, 300])
    axmat[2, i].set_ylim(- 0.2, 0.025)
    axmat[2, i].set_yticks([- 0.15, - 0.1, - 0.05, 0.0])
    axmat[2, i].set_xticks([0, 400, 800])
    axmat[2, i].set_xlabel('Steps', labelpad=X_LABEL_PAD)
    for j in range(3):
        axmat[j, i].set_xlim(0, xmax * 1.05)
        if i != 0:
            axmat[j, i].tick_params(axis='y', which='both', left=False)
            axmat[j, i].set_yticklabels([])
fig.align_ylabels(axmat[:, 0])
fig.tight_layout()
fig.subplots_adjust(hspace=0.0, wspace=0.0, right=0.84)
pos = axmat[0, 0].get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('mnist_additional_interference_momentum.pdf')

In [ ]:
plot_data = {
    r'$\beta={}$'.format(k[1]) : v
    for k, v in data.items()
    if k[0] == 'rms'}
threshold = 125
fig, axmat = plt.subplots(3, 4, sharex=True, dpi=300, figsize=(7, 4))
colors = sns.color_palette('colorblind', len(data.keys()))
xmax = 0
for j in range(4):
    for i, (k, v) in enumerate(sorted(plot_data.items(), key=lambda x: x[0])):
        other_phase = (j + 1) % 2 + 1
        x = np.where(v['d{}_count'.format(other_phase)][j] >= threshold)[0]
        xmax = max(xmax, max(x))
        y = v['d{}_avg'.format(other_phase)][j][x]
        yerr = v['d{}_sem'.format(other_phase)][j][x]
        label = k if j == 0 else None
        axmat[0, j].plot(
            x,
            y,
            linestyle='--',
            linewidth=1,
            color=colors[i])
        axmat[0, j].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

        x = np.where(v['online_count'][j] >= threshold)[0]
        xmax = max(xmax, max(x))
        y = v['online_avg'][j][x]
        yerr = v['online_sem'][j][x]
        axmat[0, j].plot(
            x,
            y,
            label=label,
            linewidth=1,
            color=colors[i])
        axmat[0, j].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

        x = np.where(v['as_count'][j] >= threshold)[0]
        xmax = max(xmax, max(x))
        y = v['as_avg'][j][x]
        yerr = v['as_sem'][j][x]
        axmat[1, j].plot(
            x,
            y,
            linewidth=1,
            color=colors[i])
        axmat[1, j].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

        x = np.where(v['pi_count'][j] >= threshold)[0]
        xmax = max(xmax, max(x))
        y = - v['pi_avg'][j][x]
        yerr = v['pi_sem'][j][x]
        axmat[2, j].plot(
            x,
            y,
            linewidth=1,
            color=colors[i])
        axmat[2, j].fill_between(
            x,
            y - yerr,
            y + yerr,
            color=colors[i],
            alpha=0.3)

axmat[0, 0].set_ylabel('Accuracy', labelpad=Y_LABEL_PAD)
axmat[1, 0].set_ylabel('Activation\nOverlap', labelpad=Y_LABEL_PAD)
axmat[2, 0].set_ylabel('Pairwise\nInterference', labelpad=Y_LABEL_PAD)
for i in range(4):
    axmat[0, i].set_title('Phase {}'.format(i + 1))
    axmat[0, i].set_ylim(-0.05, 1.05)
    axmat[0, i].set_yticks([0.0, 0.5, 1.0])
    axmat[1, i].set_ylim(- 25, 375)
    axmat[1, i].set_yticks([0, 100, 200, 300])
    axmat[2, i].set_ylim(- 0.2, 0.025)
    axmat[2, i].set_yticks([-0.15, -0.1, -0.05, 0.0])
    axmat[2, i].set_xticks([0, 50, 100])
    axmat[2, i].set_xlabel('Steps', labelpad=X_LABEL_PAD)
    for j in range(3):
        axmat[j, i].set_xlim(0, xmax * 1.05)
        if i != 0:
            axmat[j, i].tick_params(axis='y', which='both', left=False)
            axmat[j, i].set_yticklabels([])
fig.align_ylabels(axmat[:, 0])
fig.tight_layout()
fig.subplots_adjust(hspace=0.0, wspace=0.0, right=0.83)
pos = axmat[0, 0].get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('mnist_additional_interference_rms.pdf')

<a href=#top>Back to Top</a>

## Mountain Car<a name='mountain_car'/>

### Validation Phase<a name='mountain_car_validation'/>

In [ ]:
data = mc_tools.load_data(['mountain_car_validation.json'])
summary = mc_tools.get_summary(data)
best = mc_tools.get_best(data, ['optimizer', 'momentum', 'rho'], 'auc', summary=summary)
best.head(n=len(best))

In [ ]:
lr_comparison = dict()
for optimizer in set(summary['optimizer'].unique()) - {'constant'}:
    lr_comparison[optimizer] = summary[summary['optimizer'] == optimizer].sort_values('lr')
lr_comparison['momentum'] = lr_comparison['momentum'][lr_comparison['momentum']['momentum'] == 0.9]
lr_comparison['rms'] = lr_comparison['rms'][lr_comparison['rms']['rho'] == 0.999]

colors = sns.color_palette(n_colors=len(lr_comparison.keys()))
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, dpi=300, figsize=(4.5, 5.5))
sort_key = lambda x: x[0]
for i, (optimizer, value) in enumerate(sorted(lr_comparison.items(), key=sort_key)):
    y1 = value['mean_accuracy_mean'].to_numpy()
    y1err = value['mean_accuracy_stderr'].to_numpy()
    y1[np.isnan(y1)] = 95  # plot out of axis values to show divergence
    x = value['lr']
    ax1.plot(
        x,
        y1,
        '-o',
        label=optimizer_nice_names[optimizer],
        markersize=2,
        linewidth=1,
        color=colors[i])
    ax1.fill_between(
        x,
        y1 - y1err,
        y1 + y1err,
        alpha=0.3,
        color=colors[i])

    value = value[value['final_accuracy_mean'].notnull()]
    x = value['lr']
    try:
        y2 = value['mean_activation_similarity_mean']
        y2err = value['mean_activation_similarity_stderr']
        ax2.plot(
            x,
            y2,
            '-o',
            linewidth=1,
            markersize=2,
            color=colors[i])
        ax2.fill_between(
            x,
            y2 - y2err,
            y2 + y2err,
            alpha=0.3,
            color=colors[i])
    except KeyError:
        pass

    try:
        y3 = value['mean_pairwise_interference_mean']
        y3err = value['mean_pairwise_interference_stderr']
        ax3.plot(
            x,
            y3,
            '-o',
            linewidth=1,
            markersize=2,
            color=colors[i])
        ax3.fill_between(
            x,
            y3 - y3err,
            y3 + y3err,
            alpha=0.3,
            color=colors[i])
    except KeyError:
        pass

ax1.set_xscale('log', basex=2)
ax1.set_ylabel('$\overline{\mbox{RMSVE}}$', labelpad=Y_LABEL_PAD)
ax1.set_ylim(20, 90)
ax1.set_yticks([30, 60, 90])
ax2.set_ylabel('$\overline{\mbox{Activation Overlap}}$', labelpad=Y_LABEL_PAD)
ax2.set_ylim(- 25, 425)
ax2.set_yticks([0, 200, 400])
ax3.set_ylabel('$\overline{\mbox{Pairwise Interference}}$', labelpad=Y_LABEL_PAD)
ax3.set_ylim(- 425, 25)
ax3.set_yticks([- 400, - 200, 0])
ax3.set_xlim(2 ** (-18), 2 ** (-3))
ax3.set_xticks([2 ** (-18), 2 ** (-13), 2 ** (-8), 2 ** (-3)])
ax3.set_xlabel('Step-size', labelpad=X_LABEL_PAD)
fig.align_ylabels((ax1, ax2, ax3))
fig.tight_layout()
fig.subplots_adjust(hspace=0.0, right=0.725)
pos = ax1.get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('mountain_car_step-size.pdf')

<a href=#top>Back to Top</a>

### Testing Phase<a name='mountain_car_testing'/>

In [ ]:
data = mc_tools.load_data(['mountain_car_test.json'])
summary = mc_tools.get_summary(data)
summary.head(n=len(summary))

In [ ]:
rdf = summary[np.isnan(summary['momentum']) | (summary['momentum'] == 0.9)]
rdf = rdf[np.isnan(rdf['rho']) | (rdf['rho'] == 0.999)]

colors = sns.color_palette(n_colors=len(rdf.keys()))
fig, ax = plt.subplots(1, 1, sharex=True, dpi=300, figsize=(4.5, 2.5))
sort_key = lambda x: '~' if x == 'constant' else x
for i, optimizer in enumerate(sorted(rdf['optimizer'].unique(), key=sort_key)):

    if optimizer == 'constant':
        continue

    value = rdf[rdf['optimizer'] == optimizer].iloc[0].to_dict()
    y = value['accuracy_mean']
    yerr = value['accuracy_stderr']
    x = np.arange(len(y)) + 1
    ax.plot(
        x,
        y,
        label=optimizer_nice_names[optimizer],
        linewidth=1,
        color=colors[i])
    ax.fill_between(
        x,
        y - yerr,
        y + yerr,
        alpha=0.3,
        color=colors[i])

ax.set_ylabel('RMSVE', labelpad=Y_LABEL_PAD)
ax.set_ylim(20, 90)
ax.set_yticks([30, 60, 90])
ax.set_xlim(0, 500)
ax.set_xlabel('Episode', labelpad=X_LABEL_PAD)
fig.tight_layout()
fig.subplots_adjust(hspace=0.0, right=0.725)
pos = ax.get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('mountain_car_accuracies.pdf')

In [ ]:
rdf = summary[np.isnan(summary['momentum']) | (summary['momentum'] == 0.9)]
rdf = rdf[np.isnan(rdf['rho']) | (rdf['rho'] == 0.999)]

colors = sns.color_palette(n_colors=len(rdf.keys()))
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, dpi=300, figsize=(4.5, 5.5))
sort_key = lambda x: '~' if x == 'constant' else x
for i, optimizer in enumerate(sorted(rdf['optimizer'].unique(), key=sort_key)):
    value = rdf[rdf['optimizer'] == optimizer].iloc[0].to_dict()
    y1 = value['accuracy_mean']
    y1err = value['accuracy_stderr']
    x = np.arange(len(y1)) + 1
    ax1.plot(
        x,
        y1,
        label=optimizer_nice_names[optimizer],
        linewidth=1,
        color=colors[i])
    ax1.fill_between(
        x,
        y1 - y1err,
        y1 + y1err,
        alpha=0.3,
        color=colors[i])

    if optimizer == 'constant':
        continue

    try:
        y2 = value['activation_similarity_mean']
        y2err = value['activation_similarity_stderr']
        ax2.plot(
            x,
            y2,
            linewidth=1,
            color=colors[i])
        ax2.fill_between(
            x,
            y2 - y2err,
            y2 + y2err,
            alpha=0.3,
            color=colors[i])
    except KeyError:
        pass

    try:
        y3 = value['pairwise_interference_mean']
        y3err = value['pairwise_interference_stderr']
        ax3.plot(
            x,
            y3,
            linewidth=1,
            color=colors[i])
        ax3.fill_between(
            x,
            y3 - y3err,
            y3 + y3err,
            alpha=0.3,
            color=colors[i])
    except KeyError:
        pass

ax1.set_ylabel('RMSVE', labelpad=Y_LABEL_PAD)
ax1.set_ylim(20, 90)
ax1.set_yticks([30, 60, 90])
ax2.set_ylabel('Activation Overlap', labelpad=Y_LABEL_PAD)
ax2.set_ylim(- 10, 260)
ax2.set_yticks([0, 100, 200])
ax3.set_ylabel('Pairwise Interference', labelpad=Y_LABEL_PAD)
ax3.set_ylim(- 1050, 250)
ax3.set_yticks([- 1000, - 500, 0])
ax3.set_xlim(0, 500)
ax3.set_xlabel('Episode', labelpad=X_LABEL_PAD)
fig.align_ylabels((ax1, ax2, ax3))
fig.tight_layout()
fig.subplots_adjust(hspace=0.0, right=0.725)
pos = ax1.get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('mountain_car_accuracies_and_interference.pdf')

In [ ]:
rdf = summary[np.isnan(summary['momentum']) | (summary['momentum'] == 0.9)]
rdf = rdf[np.isnan(rdf['rho']) | (rdf['rho'] == 0.999)]
sort_key = lambda x: '~' if x == 'constant' else x
optimizers = sorted(rdf['optimizer'].unique(), key=sort_key)
optimizers = [optimizer for optimizer in optimizers if optimizer != 'constant']

text = ''
text += '|---------------------------|-----------|---------------|--------------|\n'
text += '|          Metric           | Optimizer |  Mean Value   | Final Value  |\n'
text += '|---------------------------|-----------|---------------|--------------|\n'
for metric in ['activation_similarity', 'pairwise_interference']:
    for i, optimizer in enumerate(optimizers):
        mean_metric_mean = rdf[rdf['optimizer'] == optimizer].iloc[0].to_dict()['mean_{}_mean'.format(metric)]
        mean_metric_stderr = rdf[rdf['optimizer'] == optimizer].iloc[0].to_dict()['mean_{}_stderr'.format(metric)]
        final_metric_mean = rdf[rdf['optimizer'] == optimizer].iloc[0].to_dict()['final_{}_mean'.format(metric)]
        final_metric_stderr = rdf[rdf['optimizer'] == optimizer].iloc[0].to_dict()['final_{}_stderr'.format(metric)]
        text += '| {0:>25} | {1:<9} | {2:>7.2f}+-{3:<4.2f} | {4:>6.2f}+-{5:<4.2f} |\n'.format(
            metric.replace('_', ' ').title() if i == 1 else '',
            optimizer_nice_names[optimizer],
            mean_metric_mean,
            mean_metric_stderr,
            final_metric_mean,
            final_metric_stderr)
    text += '|---------------------------|-----------|---------------|--------------|\n'
with open('mountain_car_metrics.txt', 'w') as outfile:
    print(text, file=outfile)
print(text)

In [ ]:
colors = sns.color_palette(n_colors=4)
fig, axmat = plt.subplots(3, 2, sharex=True, dpi=300, figsize=(5.5, 5.5))
sort_key = lambda x: '~' if x == 'constant' else x
for i, (optimizer, hyperparameter) in enumerate([('momentum', 'momentum'), ('rms', 'rho')]):
    rdf = summary[summary['optimizer'] == optimizer]
    for j, key in enumerate(['accuracy', 'activation_similarity', 'pairwise_interference']):
        hyperparameter_values = sorted(rdf[hyperparameter].unique())
        for k, hyperparameter_value in enumerate(hyperparameter_values):
            value = rdf[rdf[hyperparameter] == hyperparameter_value].iloc[0].to_dict()
            ax = axmat[j, i]
            try:
                y = value['{}_mean'.format(key)]
                yerr = value['{}_stderr'.format(key)]
                x = np.arange(len(y)) + 1
                if (i == 1) and (j == 0):
                    label = hyperparameter_value
                else:
                    label = None
                ax.plot(
                    x,
                    y,
                    label=label,
                    linewidth=1,
                    color=colors[k])
                ax.fill_between(
                    x,
                    y - yerr,
                    y + yerr,
                    alpha=0.3,
                    color=colors[k])
            except KeyError:
                pass

axmat[0, 0].set_title('Momentum')
axmat[0, 1].set_title('RMSProp')
axmat[0, 0].set_ylabel('RMSVE', labelpad=Y_LABEL_PAD)
axmat[0, 0].set_ylim(20, 90)
axmat[0, 0].set_yticks([30, 60, 90])
axmat[0, 1].set_ylim(20, 90)
axmat[0, 1].set_yticks([30, 60, 90])
axmat[1, 0].set_ylabel('Activation Overlap', labelpad=Y_LABEL_PAD)
axmat[1, 0].set_ylim(- 10, 260)
axmat[1, 0].set_yticks([0, 100, 200])
axmat[1, 1].set_ylim(- 10, 260)
axmat[1, 1].set_yticks([0, 100, 200])
axmat[2, 0].set_ylabel('Pairwise Interference', labelpad=Y_LABEL_PAD)
axmat[2, 0].set_ylim(- 1050, 250)
axmat[2, 0].set_yticks([- 1000, - 500, 0])
axmat[2, 1].set_ylim(- 1050, 250)
axmat[2, 1].set_yticks([- 1000, - 500, 0])
for i in range(2):
    axmat[2, i].set_xlim(0, 500)
    axmat[2, i].set_xlabel('Episode', labelpad=X_LABEL_PAD)
fig.align_ylabels(axmat[:, 0])
fig.tight_layout()
fig.subplots_adjust(hspace=0.0, wspace=0.375, right=0.83)
pos = axmat[0, 0].get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('mountain_car_momentum_and_rms.pdf')

<a href=#top>Back to Top</a>

## Acrobot<a name='acrobot'/>

### Validation Phase<a name='acrobot_validation'/>

In [ ]:
data = ac_tools.load_data(['acrobot_validation.json'])
summary = ac_tools.get_summary(data)
best = ac_tools.get_best(data, ['optimizer', 'momentum', 'rho'], 'auc', summary=summary)
best.head(n=len(best))

In [ ]:
lr_comparison = dict()
for optimizer in set(summary['optimizer'].unique()) - {'constant'}:
    lr_comparison[optimizer] = summary[summary['optimizer'] == optimizer].sort_values('lr')
lr_comparison['momentum'] = lr_comparison['momentum'][lr_comparison['momentum']['momentum'] == 0.9]
lr_comparison['rms'] = lr_comparison['rms'][lr_comparison['rms']['rho'] == 0.999]

colors = sns.color_palette(n_colors=len(lr_comparison.keys()))
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, dpi=300, figsize=(4.5, 5.5))
sort_key = lambda x: x[0]
for i, (optimizer, value) in enumerate(sorted(lr_comparison.items(), key=sort_key)):
    y1 = value['mean_accuracy_mean'].to_numpy()
    y1err = value['mean_accuracy_stderr'].to_numpy()
    y1[np.isnan(y1)] = 280  # plot out of axis values to show divergence
    x = value['lr']
    ax1.plot(
        x,
        y1,
        '-o',
        label=optimizer_nice_names[optimizer],
        markersize=2,
        linewidth=1,
        color=colors[i])
    ax1.fill_between(
        x,
        y1 - y1err,
        y1 + y1err,
        alpha=0.3,
        color=colors[i])

    value = value[value['final_accuracy_mean'].notnull()]
    x = value['lr']
    try:
        y2 = value['mean_activation_similarity_mean']
        y2err = value['mean_activation_similarity_stderr']
        ax2.plot(
            x,
            y2,
            '-o',
            linewidth=1,
            markersize=2,
            color=colors[i])
        ax2.fill_between(
            x,
            y2 - y2err,
            y2 + y2err,
            alpha=0.3,
            color=colors[i])
    except KeyError:
        pass

    try:
        y3 = value['mean_pairwise_interference_mean'].to_numpy()
        y3err = value['mean_pairwise_interference_stderr'].to_numpy()
        y3err[(y3 < - 2100) | (y3 > 100)] = 0  # stop standard error creeping in from out of axis values
        ax3.plot(
            x,
            y3,
            '-o',
            linewidth=1,
            markersize=2,
            color=colors[i])
        ax3.fill_between(
            x,
            y3 - y3err,
            y3 + y3err,
            alpha=0.3,
            color=colors[i])
    except KeyError:
        pass

ax1.set_xscale('log', basex=2)
ax1.set_ylabel('$\overline{\mbox{RMSVE}}$', labelpad=Y_LABEL_PAD)
ax1.set_ylim(25, 275)
ax1.set_yticks([50, 150, 250])
ax2.set_ylabel('$\overline{\mbox{Activation Overlap}}$', labelpad=Y_LABEL_PAD)
ax2.set_ylim(- 40, 240)
ax2.set_yticks([0, 100, 200])
ax3.set_ylabel('$\overline{\mbox{Pairwise Interference}}$', labelpad=Y_LABEL_PAD)
ax3.set_ylim(- 2100, 100)
ax3.set_yticks([- 2000, - 1000, 0])
ax3.set_xlim(2 ** (-18), 2 ** (-3))
ax3.set_xticks([2 ** (-18), 2 ** (-13), 2 ** (-8), 2 ** (-3)])
ax3.set_xlabel('Step-size', labelpad=X_LABEL_PAD)
fig.align_ylabels((ax1, ax2, ax3))
fig.tight_layout()
fig.subplots_adjust(hspace=0.0, right=0.725)
pos = ax1.get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('acrobot_step-size.pdf')

<a href=#top>Back to Top</a>

### Testing Phase<a name='acrobot_testing'/>

In [ ]:
data = ac_tools.load_data(['acrobot_test.json'])
summary = ac_tools.get_summary(data)
summary.head(n=len(summary))

In [ ]:
rdf = summary[np.isnan(summary['momentum']) | (summary['momentum'] == 0.9)]
rdf = rdf[np.isnan(rdf['rho']) | (rdf['rho'] == 0.999)]

colors = sns.color_palette(n_colors=len(rdf.keys()))
fig, ax = plt.subplots(1, 1, sharex=True, dpi=300, figsize=(4.5, 2.5))
sort_key = lambda x: '~' if x == 'constant' else x
for i, optimizer in enumerate(sorted(rdf['optimizer'].unique(), key=sort_key)):

    if optimizer == 'constant':
        continue

    value = rdf[rdf['optimizer'] == optimizer].iloc[0].to_dict()
    y = value['accuracy_mean']
    yerr = value['accuracy_stderr']
    x = np.arange(len(y)) + 1
    ax.plot(
        x,
        y,
        label=optimizer_nice_names[optimizer],
        linewidth=1,
        color=colors[i])
    ax.fill_between(
        x,
        y - yerr,
        y + yerr,
        alpha=0.3,
        color=colors[i])

ax.set_ylabel('RMSVE', labelpad=Y_LABEL_PAD)
ax.set_ylim(20, 100)
ax.set_yticks([30, 60, 90])
ax.set_xlim(0, 500)
ax.set_xlabel('Episode', labelpad=X_LABEL_PAD)
fig.tight_layout()
fig.subplots_adjust(hspace=0.0, right=0.725)
pos = ax.get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('acrobot_accuracies.pdf')

In [ ]:
rdf = summary[np.isnan(summary['momentum']) | (summary['momentum'] == 0.9)]
rdf = rdf[np.isnan(rdf['rho']) | (rdf['rho'] == 0.999)]

colors = sns.color_palette(n_colors=len(rdf.keys()))
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, dpi=300, figsize=(4.5, 5.5))
sort_key = lambda x: '~' if x == 'constant' else x
for i, optimizer in enumerate(sorted(rdf['optimizer'].unique(), key=sort_key)):
    value = rdf[rdf['optimizer'] == optimizer].iloc[0].to_dict()
    y1 = value['accuracy_mean']
    y1err = value['accuracy_stderr']
    x = np.arange(len(y1)) + 1
    ax1.plot(
        x,
        y1,
        label=optimizer_nice_names[optimizer],
        linewidth=1,
        color=colors[i])
    ax1.fill_between(
        x,
        y1 - y1err,
        y1 + y1err,
        alpha=0.3,
        color=colors[i])

    if optimizer == 'constant':
        continue

    try:
        y2 = value['activation_similarity_mean']
        y2err = value['activation_similarity_stderr']
        ax2.plot(
            x,
            y2,
            linewidth=1,
            color=colors[i])
        ax2.fill_between(
            x,
            y2 - y2err,
            y2 + y2err,
            alpha=0.3,
            color=colors[i])
    except KeyError:
        pass

    try:
        y3 = value['pairwise_interference_mean']
        y3err = value['pairwise_interference_stderr']
        ax3.plot(
            x,
            y3,
            linewidth=1,
            color=colors[i])
        ax3.fill_between(
            x,
            y3 - y3err,
            y3 + y3err,
            alpha=0.3,
            color=colors[i])
    except KeyError:
        pass

ax1.set_ylabel('RMSVE', labelpad=Y_LABEL_PAD)
ax1.set_ylim(20, 100)
ax1.set_yticks([30, 60, 90])
ax2.set_ylabel('Activation Overlap', labelpad=Y_LABEL_PAD)
ax2.set_ylim(0, 300)
ax2.set_yticks([50, 150, 250])
ax3.set_ylabel('Pairwise Interference', labelpad=Y_LABEL_PAD)
ax3.set_ylim(- 2100, 100)
ax3.set_yticks([- 2000, - 1000, 0])
ax3.set_xlim(0, 500)
ax3.set_xlabel('Episode', labelpad=X_LABEL_PAD)
fig.align_ylabels((ax1, ax2, ax3))
fig.tight_layout()
fig.subplots_adjust(hspace=0.0, right=0.725)
pos = ax1.get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('acrobot_accuracies_and_interference.pdf')

In [ ]:
rdf = summary[np.isnan(summary['momentum']) | (summary['momentum'] == 0.9)]
rdf = rdf[np.isnan(rdf['rho']) | (rdf['rho'] == 0.999)]
sort_key = lambda x: '~' if x == 'constant' else x
optimizers = sorted(rdf['optimizer'].unique(), key=sort_key)
optimizers = [optimizer for optimizer in optimizers if optimizer != 'constant']

text = ''
text += '|---------------------------|-----------|-----------------|----------------|\n'
text += '|          Metric           | Optimizer |   Mean Value    |  Final Value   |\n'
text += '|---------------------------|-----------|-----------------|----------------|\n'
for metric in ['activation_similarity', 'pairwise_interference']:
    for i, optimizer in enumerate(optimizers):
        mean_metric_mean = rdf[rdf['optimizer'] == optimizer].iloc[0].to_dict()['mean_{}_mean'.format(metric)]
        mean_metric_stderr = rdf[rdf['optimizer'] == optimizer].iloc[0].to_dict()['mean_{}_stderr'.format(metric)]
        final_metric_mean = rdf[rdf['optimizer'] == optimizer].iloc[0].to_dict()['final_{}_mean'.format(metric)]
        final_metric_stderr = rdf[rdf['optimizer'] == optimizer].iloc[0].to_dict()['final_{}_stderr'.format(metric)]
        text += '| {0:>25} | {1:<9} | {2:>8.2f}+-{3:<5.2f} | {4:>7.2f}+-{5:<5.2f} |\n'.format(
            metric.replace('_', ' ').title() if i == 1 else '',
            optimizer_nice_names[optimizer],
            mean_metric_mean,
            mean_metric_stderr,
            final_metric_mean,
            final_metric_stderr)
    text += '|---------------------------|-----------|-----------------|----------------|\n'
with open('acrobot_metrics.txt', 'w') as outfile:
    print(text, file=outfile)
print(text)

In [ ]:
colors = sns.color_palette(n_colors=4)
fig, axmat = plt.subplots(3, 2, sharex=True, dpi=300, figsize=(5.5, 5.5))
sort_key = lambda x: '~' if x == 'constant' else x
for i, (optimizer, hyperparameter) in enumerate([('momentum', 'momentum'), ('rms', 'rho')]):
    rdf = summary[summary['optimizer'] == optimizer]
    for j, key in enumerate(['accuracy', 'activation_similarity', 'pairwise_interference']):
        hyperparameter_values = sorted(rdf[hyperparameter].unique())
        for k, hyperparameter_value in enumerate(hyperparameter_values):
            value = rdf[rdf[hyperparameter] == hyperparameter_value].iloc[0].to_dict()
            ax = axmat[j, i]
            try:
                y = value['{}_mean'.format(key)]
                yerr = value['{}_stderr'.format(key)]
                if key == 'accuracy':
                    y[np.isnan(y)] = 105
                x = np.arange(len(y)) + 1
                if (i == 1) and (j == 0):
                    label = hyperparameter_value
                else:
                    label = None
                ax.plot(
                    x,
                    y,
                    label=label,
                    linewidth=1,
                    color=colors[k])
                ax.fill_between(
                    x,
                    y - yerr,
                    y + yerr,
                    alpha=0.3,
                    color=colors[k])
            except KeyError:
                pass

axmat[0, 0].set_title('Momentum')
axmat[0, 1].set_title('RMSProp')
axmat[0, 0].set_ylabel('RMSVE', labelpad=Y_LABEL_PAD)
axmat[0, 0].set_ylim(20, 100)
axmat[0, 0].set_yticks([30, 60, 90])
axmat[0, 1].set_ylim(20, 100)
axmat[0, 1].set_yticks([30, 60, 90])
axmat[1, 0].set_ylabel('Activation Overlap', labelpad=Y_LABEL_PAD)
axmat[1, 0].set_ylim(0, 300)
axmat[1, 0].set_yticks([50, 150, 250])
axmat[1, 1].set_ylim(0, 300)
axmat[1, 1].set_yticks([50, 150, 250])
axmat[2, 0].set_ylabel('Pairwise Interference', labelpad=Y_LABEL_PAD)
axmat[2, 0].set_ylim(- 2100, 100)
axmat[2, 0].set_yticks([- 2000, - 1000, 0])
axmat[2, 1].set_ylim(- 2100, 100)
axmat[2, 1].set_yticks([- 2000, - 1000, 0])
for i in range(2):
    axmat[2, i].set_xlim(0, 500)
    axmat[2, i].set_xlabel('Episode', labelpad=X_LABEL_PAD)
fig.align_ylabels(axmat[: ,0])
fig.tight_layout()
fig.subplots_adjust(hspace=0.0, wspace=0.375, right=0.83)
pos = axmat[0, 0].get_position()
fig.legend(bbox_to_anchor=(1, pos.y0 + (pos.y1 - pos.y0) / 2), loc='center right')
fig.savefig('acrobot_momentum_and_rms.pdf')

<a href=#top>Back to Top</a>

## Summary<a name='summary'/>

In [ ]:
mc_data = mc_tools.load_data(['mountain_car_test.json'])
mc_summary = mc_tools.get_summary(mc_data)
mc_summary.head(n=len(mc_summary))

ac_data = ac_tools.load_data(['acrobot_test.json'])
ac_summary = ac_tools.get_summary(ac_data)
ac_summary.head(n=len(ac_summary))

# setup matplotlib
sns.set_style('ticks')
sns.set_context('notebook')
sns.set_style('ticks')
sns.set_context('notebook')

In [ ]:
colors = sns.color_palette(n_colors=len(mc_summary['optimizer'].unique()) - 1)
fig, axmat = plt.subplots(2, 2, sharex=True, dpi=300, figsize=(6, 4.5))
sort_key = lambda x: '~' if x == 'constant' else x
for i, (setting, summary) in enumerate([('Mountain Car', mc_summary), ('Acrobot', ac_summary)]):
    rdf = summary[np.isnan(summary['momentum']) | (summary['momentum'] == 0.9)]
    rdf = rdf[np.isnan(rdf['rho']) | (rdf['rho'] == 0.999)]
    rdf = rdf[rdf['optimizer'] != 'constant']
    for j, optimizer in enumerate(sorted(rdf['optimizer'].unique(), key=sort_key)):
        value = rdf[rdf['optimizer'] == optimizer].iloc[0].to_dict()
        try:
            y2 = value['activation_similarity_mean']
            y2err = value['activation_similarity_stderr']
            x = np.arange(len(y2)) + 1
            axmat[0, i].plot(
                x,
                y2,
                label=optimizer_nice_names[optimizer] if i == 0 else None,
                linewidth=1,
                color=colors[j])
            axmat[0, i].fill_between(
                x,
                y2 - y2err,
                y2 + y2err,
                alpha=0.3,
                color=colors[j])
        except KeyError:
            pass
        try:
            y3 = value['pairwise_interference_mean']
            y3err = value['pairwise_interference_stderr']
            x = np.arange(len(y3)) + 1
            axmat[1, i].plot(
                x,
                y3,
                linewidth=1,
                color=colors[j])
            axmat[1, i].fill_between(
                x,
                y3 - y3err,
                y3 + y3err,
                alpha=0.3,
                color=colors[j])
        except KeyError:
            pass

axmat[0, 0].set_ylabel('Activation Overlap', labelpad=10)
axmat[1, 0].set_ylabel('Pairwise Interference', labelpad=10)
axmat[0, 0].set_ylim(- 10, 260)
axmat[0, 0].set_yticks([0, 100, 200])
axmat[1, 0].set_ylim(- 1050, 250)
axmat[1, 0].set_yticks([- 1000, - 500, 0])
axmat[0, 1].set_ylim(- 5, 105)
axmat[0, 1].set_yticks([0, 50, 100])
axmat[1, 1].set_ylim(- 2100, 100)
axmat[1, 1].set_yticks([- 2000, - 1000, 0])
for i in range(2):
    axmat[0, i].set_xlim(0, 500)
    axmat[1, i].set_xlim(0, 500)
    axmat[1, i].set_xticks([0, 200, 400])
    axmat[1, i].set_xlabel('Episode', labelpad=10)
axmat[0, 0].set_title('Mountain Car')
axmat[0, 1].set_title('Acrobot')
lgd = axmat[0, 0].legend(loc='upper right', bbox_to_anchor=(3.125, 0.85), frameon=False)
fig.align_ylabels(axmat[:, 0])
fig.subplots_adjust(hspace=0.0, wspace=0.375)
fig.savefig('experiment_2_mountain_car_and_acrobot_interference.pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
x = sorted(list(set(optimizer_nice_names.values()) - {'Constant'}))
y = [1, 4, 5, 6]
fig, ax = plt.subplots(1, 1, dpi=300, figsize=(3.5, 2))
ax.bar(x,
       y,
       0.6,
       color=sns.color_palette('colorblind', 1)[0])
ax.set_yticks([0, 2, 4, 6, 8])
ax.set_ylim(0, 8)
ax.set_ylabel('Number of Times in Top 2', labelpad=10)
fig.tight_layout()
fig.savefig('top_2.pdf')

<a href=#top>Back to Top</a>